# Getting Started with ESA++

ESA++ wraps PowerWorld's SimAuto COM interface into Pythonic indexing.
The core abstraction is the **indexable interface** — you read and write
grid data using familiar bracket notation (`wb[Type, fields]`), and
every result comes back as a pandas DataFrame.

```python
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch, Shunt, Area, Zone

wb = GridWorkBench("path/to/case.pwb")
```

In [53]:
from esapp import GridWorkBench
from esapp.components import Bus, Gen, Load, Branch, Shunt, Area, Zone
from esapp import TS
from esapp.components import TSField
import numpy as np
import pandas as pd
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

'open' took: 14.3333 sec


## Reading Data

The indexable interface supports four read patterns, all accessed
through `wb[...]`. Every query returns a pandas DataFrame with
primary key columns automatically included.

| Syntax | What you get |
|---|---|
| `wb[Bus]` | Key columns only (e.g. `BusNum`) |
| `wb[Bus, "BusPUVolt"]` | Keys + one field |
| `wb[Bus, ["BusPUVolt", "BusAngle"]]` | Keys + multiple fields |
| `wb[Bus, :]` | Keys + **every** defined field |

Passing just the component type returns its primary key columns. This
is useful when you need to know which objects exist. Generators have a
compound key (`BusNum`, `GenID`), while buses are identified by
`BusNum` alone.

In [54]:
wb[Bus].head()

,BusNum
0,1
1,2
2,3
3,4
4,5


In [55]:
wb[Gen].head()

,BusNum,GenID
0,2,1
1,2,2
2,2,3
3,2,4
4,23,1


Adding a field name returns that column alongside the keys. A list
requests several columns at once.

In [56]:
wb[Bus, "BusPUVolt"].head()

,BusNum,BusPUVolt
0,1,0.993545
1,2,0.991225
2,3,0.984548
3,4,0.978800
4,5,0.988985


In [57]:
wb[Gen, ["GenMW", "GenMVR", "GenStatus"]].head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


Instead of raw strings you can use the component's enum attributes,
which gives you autocomplete in your IDE and catches typos at
definition time rather than at runtime.

In [58]:
wb[Bus, [Bus.BusName, Bus.BusPUVolt, Bus.BusAngle]].head()

,BusAngle,BusName,BusNum,BusPUVolt
0,-1.119907,ALOHA138,1,0.993545
1,-3.927372,ALOHA69,2,0.991225
2,-4.731145,FLOWER69,3,0.984548
3,-5.745870,WAVE69,4,0.978800
4,-2.069792,HONOLULU138,5,0.988985


The slice syntax `wb[Type, :]` retrieves every field defined for a
component. This is handy for exploration but can return wide
DataFrames.

In [59]:
buses = wb[Bus, :]
buses.shape

(37, 581)

Every result is a standard pandas DataFrame, so all the usual
filtering and aggregation tools apply.

In [60]:
gens = wb[Gen, ["GenMW", "GenMVR", "GenStatus"]]
gens[gens["GenStatus"] == "Closed"].head()

,BusNum,GenID,GenMVR,GenMW,GenStatus
0,2,1,0.80000,2.500000,Closed
1,2,2,0.80000,2.500000,Closed
2,2,3,0.80000,2.500000,Closed
3,2,4,0.80000,2.500000,Closed
4,23,1,0.04408,69.274741,Closed


In [61]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
loads[["LoadMW", "LoadMVR"]].sum()

LoadMW     1136.290004
LoadMVR       0.000000
dtype: float64

## Writing Data

The same bracket syntax supports writes.

| Syntax | Behavior |
|---|---|
| `wb[Gen, "GenMW"] = 100.0` | Broadcast a **scalar** to every object |
| `wb[Gen, "GenMW"] = [100, 150, ...]` | Set **per-element** values (length must match) |
| `wb[Gen, ["GenMW", "GenStatus"]] = [100, "Closed"]` | Broadcast to **multiple fields** at once |
| `wb[Bus] = df` | **Bulk update** from a DataFrame (must include key columns) |

A single value is broadcast to every object of that type.

In [62]:
wb[Gen, "GenMW"] = 100.0
wb[Gen, "GenMW"].head()

,BusNum,GenID,GenMW
0,2,1,100.0
1,2,2,100.0
2,2,3,100.0
3,2,4,100.0
4,23,1,100.0


A list or array whose length matches the number of existing objects
sets each one individually. Multiple fields can be set in a single
call as well.

In [63]:
wb[Gen, "GenMW"] = np.linspace(50, 200, len(wb[Gen]))
wb[Gen, "GenMW"].head()

,BusNum,GenID,GenMW
0,2,1,50.000000
1,2,2,53.409094
2,2,3,56.818181
3,2,4,60.227275
4,23,1,63.636363


In [64]:
wb[Gen, ["GenMW", "GenStatus"]] = [100.0, "Closed"]
wb[Gen, ["GenMW", "GenStatus"]].head()

,BusNum,GenID,GenMW,GenStatus
0,2,1,100.0,Closed
1,2,2,100.0,Closed
2,2,3,100.0,Closed
3,2,4,100.0,Closed
4,23,1,100.0,Closed


For targeted updates to specific objects, build a DataFrame that
includes the primary key columns and any fields you want to change,
then assign it directly.

In [65]:
updates = pd.DataFrame({
    "BusNum": wb[Bus]["BusNum"].head(3),
    "BusPUVolt": [1.02, 1.01, 0.99]
})
wb[Bus] = updates

wb[Bus, "BusPUVolt"].head(3)

,BusNum,BusPUVolt
0,1,1.02
1,2,1.01
2,3,0.99


A common workflow is to read existing values, transform them in
pandas, and write the result back. Here we scale all loads up by 10%.

In [66]:
loads = wb[Load, ["LoadMW", "LoadMVR"]]
loads["LoadMW"] *= 1.10
loads["LoadMVR"] *= 1.10
wb[Load] = loads

wb[Load, ["LoadMW", "LoadMVR"]][["LoadMW", "LoadMVR"]].sum()

LoadMW     1249.919015
LoadMVR       0.000000
dtype: float64

## Component Metadata

Every component class carries metadata about its fields that controls
how the indexable interface behaves during reads and writes.

**Primary keys** uniquely identify an object in PowerWorld. These are
always included in read results and are required when writing a
DataFrame back. Buses have a single key (`BusNum`), while generators
and branches use compound keys.

In [67]:
Bus.keys, Gen.keys, Branch.keys

(['BusNum'],
 ['BusNum', 'GenID'],
 ['BusName_NomVolt:1', 'BusNum', 'LineCircuit', 'BusNum:1'])

**Identifiers** are the union of primary and secondary keys. Secondary
keys are fields like `BusName` or `AreaNum` that PowerWorld uses
alongside primary keys to locate or describe objects. When you write
data, any identifier column can be included in your DataFrame — they
help PowerWorld resolve which object you mean.

In [68]:
Bus.identifiers

{'AreaNum', 'BusName', 'BusName_NomVolt', 'BusNomVolt', 'BusNum', 'ZoneNum'}

**Editable** fields are the ones you can modify on existing objects —
generation setpoints, voltage targets, load values, and so on. The
indexable interface validates this: attempting to write a read-only
field raises a `ValueError`.

In [69]:
Bus.editable[:10]

['AreaNum',
 'BusName',
 'BusNomVolt',
 'ZoneNum',
 'AllLabels',
 'AreaName',
 'BAName',
 'BANumber',
 'BusAngle',
 'BusB:1']

**Settable** is the full set of fields that can appear in a write
operation: identifiers plus editable fields. This is what the
indexable interface checks against when you assign a DataFrame — every
column must be settable, or the write is rejected. In practice this
means your DataFrames can include key columns (to identify rows) and
editable columns (to change values), but not computed or read-only
fields.

In [70]:
len(Bus.settable), len(Bus.fields)

(113, 581)

## Transient Stability Fields

The `TS` class provides a parallel set of constants for transient
stability result fields. These are organized by object type —
`TS.Gen`, `TS.Bus`, `TS.Branch`, `TS.Load`, etc. — and each field
is a `TSField` with a `name` (the PowerWorld string) and a
`description`.

```python
from esapp import TS
from esapp.components import TSField
```

Typing `TS.Gen.` in your IDE will autocomplete every available
generator TS result field. The object types mirror the component
classes.

In [71]:
TS.Gen.P, TS.Gen.Q, TS.Gen.W, TS.Gen.Delta

(TSField('TSGenP'),
 TSField('TSGenQ'),
 TSField('TSGenW'),
 TSField('TSGenDelta'))

Each `TSField` carries a description from PowerWorld's schema, which
is useful when you encounter an unfamiliar field name.

In [72]:
TS.Gen.Delta.description

'Rotor Angle relative to angle reference (degrees)'

In [73]:
TS.Bus.VPU, TS.Bus.VPU.description

(TSField('TSBusVPU'), 'Voltage Magnitude (pu)')

Some TS fields are indexed — for example, a bus can have multiple
input signals. The bracket operator creates the indexed variant,
which maps to PowerWorld's `TSBusInput:1`, `TSBusInput:2`, etc.

In [74]:
TS.Bus.Input[1], TS.Bus.Input[2]

(TSField('TSBusInput:1'), TSField('TSBusInput:2'))

To see all available fields for a given object type, use `dir()` or
inspect the class attributes directly. The inner classes on `TS`
cover `Area`, `Branch`, `Bus`, `Gen`, `InjectionGroup`, `Load`,
`Shunt`, `Substation`, and `System`.

In [75]:
[f for f in dir(TS.Gen) if not f.startswith('_')][:15]

['AGCInput',
 'AGCOther',
 'AGCState',
 'AGCStatus',
 'AeroInput',
 'AeroOther',
 'AeroState',
 'AppImpR',
 'AppImpX',
 'Delta',
 'DeltaNoshift',
 'EField',
 'ExciterInput',
 'ExciterName',
 'ExciterOther']

## Power Flow

`pflow()` solves the AC power flow and returns complex bus voltages.
After solving, the indexable interface reflects the updated state in
PowerWorld — bus voltages, generator dispatch, and branch flows all
change to match the solved case.

In [76]:
V = wb.pflow()
V.head()

0    0.952238+0.244083j
1    0.994631+0.121188j
2    0.986073+0.004044j
3    0.979584+0.057489j
4    0.954998+0.202453j
dtype: complex128

In [77]:
np.abs(V).describe()

count    37.000000
mean      1.000560
std       0.028340
min       0.969866
25%       0.979828
50%       0.986082
75%       1.021444
max       1.092867
dtype: float64

In [78]:
wb[Bus, ["BusPUVolt", "BusAngle"]].head()

,BusAngle,BusNum,BusPUVolt
0,14.376859,1,0.983023
1,6.946795,2,1.001987
2,0.234963,3,0.986082
3,3.358680,4,0.981269
4,11.969104,5,0.976222


In [79]:
wb[Gen, ["GenMW", "GenMVR"]].head()

,BusNum,GenID,GenMVR,GenMW
0,2,1,-0.500000,100.000000
1,2,2,-0.500000,100.000000
2,2,3,-0.500000,100.000000
3,2,4,-0.500000,100.000000
4,23,1,101.475453,-191.364491


In [80]:
wb[Branch, ["BusNum", "BusNum:1", "LineMVA", "LinePercent"]].sort_values("LinePercent", ascending=False).head()

,BusName_NomVolt:1,BusNum,BusNum:1,LineCircuit,LineMVA,LinePercent
72,COGEN69_69.00,24,36,1,378.298748,776.794162
67,SCHOFIELD69_69.00,23,34,1,330.785254,444.603826
68,SCHOFIELD69_69.00,23,34,2,330.785254,444.603826
19,EWA BEACH69_69.00,2,26,2,203.156863,329.265589
18,EWA BEACH69_69.00,2,26,1,203.156863,329.265589
